# Comparaison de la sélection de variables avec STABL (Random Forest, XGBoost, Lasso, ElasticNet)

Ce notebook compare la sélection de variables et les performances de classification/régression entre STABL utilisant des modèles d’arbres (Random Forest, XGBoost) avec grid search, et STABL utilisant des modèles linéaires (Lasso, ElasticNet). Les résultats sont évalués par validation croisée, en termes de performances et de nombre de variables sélectionnées.

In [9]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, Lasso, ElasticNet
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error
from stabl.stabl import Stabl
from stabl.adaptive import ALasso, ALogitLasso
from stabl.preprocessing import LowInfoFilter
# from stabl.visualization import plot_fdr_graph
from sklearn.feature_selection import VarianceThreshold
from sklearn.base import clone
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


## 1. Charger et explorer les données

Nous allons charger un jeu de données tabulaire (exemple : CyTOF.csv et outcome.csv), afficher ses dimensions, types de variables, et quelques statistiques descriptives.

In [10]:
# Charger les données (exemple : CyTOF.csv et outcome.csv)
features = pd.read_csv("data/COVID-19/Training/Proteomics.csv")
outcome = pd.read_csv("data/COVID-19/Training/Mild&ModVsSevere.csv")

print("Features shape:", features.shape)
print("Outcome shape:", outcome.shape)
print("Features columns:", features.columns.tolist()[:10], "...")
print("Outcome columns:", outcome.columns.tolist())

# Statistiques descriptives
print(features.describe())
print(outcome.describe())

# Afficher la distribution de la variable cible (exemple : model1b)
outcome["Mild&ModVsSevere"].value_counts().plot(kind="bar")
plt.title("Distribution de la variable cible")
plt.show()

Features shape: (68, 1464)
Outcome shape: (68, 2)
Features columns: ['sampleID', 'AARSD1', 'ABHD14B', 'ABL1', 'ACAA1', 'ACAN', 'ACE2', 'ACOX1', 'ACP5', 'ACP6'] ...
Outcome columns: ['sampleID', 'Mild&ModVsSevere']
          AARSD1    ABHD14B       ABL1      ACAA1       ACAN       ACE2  \
count  68.000000  68.000000  68.000000  68.000000  68.000000  68.000000   
mean    5.855332   3.636771   6.350316   5.306296   4.008700   2.594200   
std     1.122131   1.182992   1.639027   1.701369   0.357109   0.782309   
min     3.085600   1.259200   2.484700   1.735500   3.120300   1.322200   
25%     5.135525   2.712275   4.993575   4.035725   3.798150   2.003275   
50%     5.880950   3.886500   6.311700   5.555200   4.012800   2.489750   
75%     6.744550   4.626100   7.616100   6.483175   4.222925   2.996000   
max     7.848400   5.711400   9.761300   9.620800   4.938000   5.475900   

           ACOX1       ACP5       ACP6      ACTA2  ...      WNT9A       WWP2  \
count  68.000000  68.000000  6

## 2. Définir le préprocessing

Pipeline de préprocessing : imputation, standardisation, filtrage faible variance, filtrage low info.

## Configuration des pipelines STABL avec modèles d’arbres et linéaires

Dans cette section, nous allons configurer plusieurs pipelines de sélection de variables et de modélisation :
- **STABL + Random Forest** : sélection de variables avec STABL utilisant Random Forest comme estimateur, avec recherche de grille (grid search) pour optimiser les hyperparamètres, et validation croisée (cross-validation).
- **STABL + XGBoost** : même principe, mais avec XGBoost comme estimateur.
- **STABL + Lasso** : pipeline linéaire utilisant Lasso.
- **STABL + ElasticNet** : pipeline linéaire utilisant ElasticNet.

Pour chaque pipeline, nous appliquerons la sélection de variables, puis nous évaluerons les performances en cross-validation et le nombre de variables sélectionnées. Les résultats seront comparés à la fin du notebook.

## Cross-validation, sauvegarde des résultats et visualisation

Pour chaque modèle, nous allons :
- Effectuer une cross-validation (stratifiée) sur le dataset.
- Sauvegarder les courbes ROC et PR dans le dossier `Benchmarks results/` avec le nom du modèle et du dataset.
- Sauvegarder les importances des variables sélectionnées et leur nombre.
- Comparer les performances (AUC, accuracy, etc.) et le nombre de variables sélectionnées entre tous les modèles.

Les résultats seront visualisés sous forme de tableaux et de graphes pour faciliter la comparaison.

## Conclusion

Ce notebook permet de comparer la sélection de variables et les performances de classification entre STABL (Random Forest, XGBoost, Lasso, ElasticNet) sur le dataset CyTOF. Les courbes ROC, PR, importances des variables et tableaux de résultats sont sauvegardés dans le dossier `Benchmarks results/` avec des noms explicites pour chaque modèle et dataset. Vous pouvez adapter ce pipeline à d'autres datasets en modifiant la section de chargement des données.

In [11]:
# Pipeline de préprocessing (à utiliser dans chaque pipeline)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from stabl.preprocessing import LowInfoFilter

preprocessing = Pipeline([
    ("variance_threshold", VarianceThreshold(threshold=0)),
    ("low_info_filter", LowInfoFilter(max_nan_fraction=0.2)),
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

In [12]:
df = pd.read_csv("data/Biobank SSI/Proteomics.csv")
print(len(df["sampleID"].unique()))


91


In [10]:
# ===============================
# 1. Imports et configuration générale
# ===============================
import os, re
import shutil
import numpy as np
import pandas as pd
from stabl import data
from stabl.multi_omic_pipelines import multi_omic_stabl_cv, multi_omic_stabl_cv_noe
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from stabl.stabl import Stabl
from stabl.adaptive import ALogitLasso
from sklearn.base import clone
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit

np.random.seed(42)

# ===============================
# 2. Définition des splits de validation croisée
# ===============================
# Outer CV pour l'évaluation globale, inner CV pour la recherche d'hyperparamètres
#outer_cv = GroupShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

# ===============================
# 3. Définition des estimateurs et grilles d'hyperparamètres
# ===============================
artificial_type = "knockoff"  # ou "random_permutation"

# Lasso
lasso = LogisticRegression(penalty="l1", class_weight="balanced", max_iter=int(1e6), solver="liblinear", random_state=42)
lasso_cv = GridSearchCV(lasso, param_grid={"C": np.logspace(-2, 2, 30)}, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

#ElasticNet
en = LogisticRegression(penalty='elasticnet', solver='saga', class_weight='balanced', max_iter=int(1e4), random_state=42)
en_params = {"C": np.logspace(-2, 1, 10), "l1_ratio": [0.5, 0.7, 0.9]}
en_cv = GridSearchCV(en, param_grid=en_params, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# RandomForest
rf = RandomForestClassifier(random_state=42, max_features=0.2)
rf_grid = {"max_depth": [3, 5, 7, 9, 11]}
rf_cv = GridSearchCV(rf, scoring='roc_auc', param_grid=rf_grid, cv=inner_cv, n_jobs=-1)

# XGBoost
xgb = XGBClassifier(random_seed=42,n_jobs=-1)
xgb_grid = {"max_depth": [3, 6, 9], "reg_alpha": [0, 0.1, 0.5, 1]}
xgb_cv = GridSearchCV(xgb, scoring='roc_auc', param_grid=xgb_grid, cv=inner_cv, n_jobs=-1)

# CatBoost
# cb = CatBoostClassifier(random_state=42, iterations=500, verbose=False)
# cb_grid = {"depth": [3, 5, 7], "learning_rate": [0.01, 0.1, 0.2], "l2_leaf_reg": [1, 3, 5]}
# cb_cv = GridSearchCV(cb, scoring='roc_auc', param_grid=cb_grid, cv=inner_cv, n_jobs=-1)

# ===============================
# 4. Définition des estimateurs STABL
# ===============================
stabl_lasso = Stabl(
    base_estimator=lasso,
    n_bootstraps=100,
    artificial_type=artificial_type,
    artificial_proportion=1.,
    replace=False,
    fdr_threshold_range=np.arange(0.1, 1, 0.01),
    sample_fraction=0.5,
    random_state=42,
    lambda_grid={"C": np.linspace(0.01, 1, 10)},
    verbose=1
)

stabl_en = clone(stabl_lasso).set_params(
    base_estimator=en,
    n_bootstraps=100,
    lambda_grid=[{"C": np.logspace(-2, 1, 5), "l1_ratio": [0.5, 0.9]}],
    verbose=1
)

stabl_rf = clone(stabl_lasso).set_params(
    base_estimator=rf,
    n_bootstraps=100,
    lambda_grid=rf_grid,
    verbose=1
)

stabl_xgb = clone(stabl_lasso).set_params(
    base_estimator=xgb,
    #bootstrap_threshold="median",
    n_bootstraps=100,
    lambda_grid=[xgb_grid],
    verbose=1
)

# stabl_cb = clone(stabl_lasso).set_params(
#     base_estimator=cb,
#     n_bootstraps=100,
#     lambda_grid=[cb_grid],
#     verbose=1
# )

# ===============================
# 5. Dictionnaire des estimateurs pour le benchmark
# ===============================
estimators = {
    "lasso": lasso_cv,
#    "alasso": alasso_cv,
    "en": en_cv,
    "xgb": xgb_cv,
    "stabl_lasso": stabl_lasso,
#    "stabl_alasso": stabl_alasso,
    "stabl_en": stabl_en,
    "stabl_xgb": stabl_xgb,
}

models = [
#    "STABL Lasso",
    "STABL ALasso",
#    "STABL ElasticNet",
    "STABL XGBoost"
]

# juste après avoir construit ton dict estimators, ajoute :
estimators["en"]        = estimators["lasso"]        # placeholder vide
estimators["stabl_en"]  = estimators["stabl_lasso"]  # placeholder vide

estimators["alasso"]        = estimators["lasso"]        # placeholder vide
estimators["stabl_alasso"]  = estimators["stabl_lasso"]  # placeholder vide


# ===============================
# 6. Chargement des données
# ===============================
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_covid_19("data/COVID-19")
X_train, X_valid, y_train, y_valid, ids, task_type = data.load_ssi("data/Biobank SSI")
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_cfrna("data/CFRNA")

print("X_train shape:", X_train)
print("y_train shape:", y_train)

# Assainir les labels
y = y_train.astype(int)
vals = sorted(set(y.unique()))
print("Labels uniques:", vals)  # doit afficher [0, 1]
assert set(vals) <= {0, 1}

# 3) Early-fusion: concat CyTOF + Proteomics en un seul X
cyto = X_train["CyTOF"]
prot = X_train["Proteomics"].copy()

# (Option recommandé) Préfixer la protéomique pour éviter toute collision de noms :
#prot.columns = [f"PRO_{c}" for c in prot.columns]

# 4) Aligner les index sur l’intersection commune (y, cyto, prot)
common = y.index
for df in (cyto, prot):
    common = common.intersection(df.index)

y_train = y.loc[common]
X_train = {
    "CyTOF": cyto.loc[common],
    "Proteomics": prot.loc[common],
}
# # 2. Isoler le gros DataFrame CyTOF + DataFrame Proteomics
# cytof_full = X_train.pop("CyTOF")          # retire temporairement
# prot_df    = X_train.pop("Proteomics")     # retire temporairement

# # 3. Décomposer CyTOF en 3 sous-ensembles
# stim_map = {"unstim": "Unstim", "lps": "LPS", "il246": "IL246", "tnf": "TNF"}
# X_train  = {}                              # on reconstruit le dict final

# for short, nice in stim_map.items():
#     regex = re.compile(fr"(^|_)({short})(_|$)", flags=re.IGNORECASE)
#     cols  = [c for c in cytof_full.columns if regex.search(c)]
#     if not cols:
#         raise ValueError(f"Aucune colonne pour la stimulation « {short} »")
#     X_train[nice] = cytof_full[cols].copy()

# # 4. Ajouter Proteomics comme 4ᵉ entrée
# X_train["Proteomics"] = prot_df.copy()

# 5. Garder UNIQUEMENT les échantillons communs aux 4 DataFrames + y
# common = y_train.index.copy()
# for df in X_train.values():
#     common = common.intersection(df.index)
# if common.empty:
#     raise ValueError("Aucun échantillon commun entre omiques et labels.")

# for k in X_train:
#     X_train[k] = X_train[k].loc[common].copy()
# y_train = y_train.loc[common]

# print({k: df.shape for k, df in X_train.items()})  # aperçu des shapes

# 6. Groupes & CV
groups    = ids

# =========================================================================
# 5. Lancement du benchmark
# =========================================================================
save_path = "./Benchmarks results/SSI + XGB_test/KO"
if os.path.exists(save_path):
    shutil.rmtree(save_path, ignore_errors=True)

print("Run CV on Biobank SSI dataset")
print("IDs :", ids)

multi_omic_stabl_cv_noe(
    data_dict=X_train,
    y=y_train,
    outer_splitter=outer_cv,
    estimators=estimators,
    task_type=task_type,
    save_path=save_path,
    model_chosen="xgboost",
    importance_method="gain",
    outer_groups=groups,
    early_fusion=False,
    late_fusion=False,
    n_iter_lf=1000,
    models=models
)

X_train shape: {'CyTOF':             unstim_Baso_149Sm_CREB  unstim_Baso_150Nd_STAT5  \
sampleID                                                      
BBCR0002                  0.632924                 0.344065   
BBCR0003                  0.921823                 0.677804   
BBCR0005-1                0.570141                 0.342670   
BBCR0006                  0.784137                 0.542236   
BBCR0007                  0.915210                 0.472476   
...                            ...                      ...   
BBCR0319                  0.620614                 0.641094   
BBCR0327                  0.354008                 0.456189   
BBCR0328                  0.475081                 0.480226   
BBCR0329                  0.286093                 0.425288   
BBCR0338                  0.528063                 0.505472   

            unstim_Baso_151Eu_p38  unstim_Baso_153Eu_STAT1  \
sampleID                                                     
BBCR0002                 0.0280

{'STABL ALasso': sampleID
 BBCR0002      0.035374
 BBCR0003      0.061681
 BBCR0005-1    0.004340
 BBCR0006      0.017272
 BBCR0007      0.009390
                 ...   
 BBCR0319      0.035850
 BBCR0327      0.003284
 BBCR0328      0.004154
 BBCR0329      0.302069
 BBCR0338      0.167790
 Length: 91, dtype: float32,
 'STABL XGBoost': sampleID
 BBCR0002      0.006207
 BBCR0003      0.020681
 BBCR0005-1    0.010045
 BBCR0006      0.001379
 BBCR0007      0.022416
                 ...   
 BBCR0319      0.021057
 BBCR0327      0.019613
 BBCR0328      0.037079
 BBCR0329      0.182680
 BBCR0338      0.018442
 Length: 91, dtype: float64}

In [42]:
X_train, X_valid, y_train, y_valid, ids, task_type = data.load_ssi("data/Biobank SSI")


X=pd.concat(X_train.values(), axis=1)

y=pd.concat([y_train], axis=1)

print("X shape:", X.shape)
print("y shape:", y.shape)


indices_X = set(X.index)
indices_y = set(y.index)

print("Index dans X_train mais pas dans y_train :", sorted(indices_X - indices_y))
print("Index dans y_train mais pas dans X_train :", sorted(indices_y - indices_X))


common = set(y.index)

common = pd.Index(sorted(common))
print("Index communs :", len(common))

xgb =XGBClassifier(
    n_estimators=1000,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
    importance_type="gain"
)

xgb.fit(X,y)

# Afficher les importances des caractéristiques
importances = xgb.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print(importance_df.head(10))



X shape: (93, 1846)
y shape: (93, 1)
Index dans X_train mais pas dans y_train : []
Index dans y_train mais pas dans X_train : []
Index communs : 93
                        Feature  Importance
1344                      PSMB9    0.056320
1484                     EIF4G1    0.042948
1327                     ASRGL1    0.034495
37    unstim_CD4Tcm_154Sm_STAT3    0.033057
351    unstim_Treg_168Er_pSTAT6    0.032746
1725                      IFNL3    0.026527
73      unstim_CD4Trm_164Dy_IkB    0.026478
1387                      CNTN3    0.025869
1649                        RAN    0.025824
1555                       CCL3    0.025051
